In [1]:
import os
import math
import numpy as np
import sympy as sp
from enum import Enum

import ContinuumMechanics as CM

Prepare Input Variables


In [2]:
L = sp.Symbol('L', positive=True)
E = sp.Symbol('E', positive=True)
A = sp.Symbol('A', positive=True)
S = E * A



Set up the nodes. Node Id will determine the position in the System Matrix.
Unlike where the system is formatted in \[\[Kcc,Kco\],\[Koc,Koo\]\] which would mean that the order matters. With this solver it does not. As it does the splitting in the "SplitSystemMatrix" function. 

![Example](BridgeExample.drawio.svg "System Example")

In [3]:
Nodes = [
    CM.Node2d(0, "Node_1", 0, L),
    CM.Node2d(1, "Node_2", L, 2*L),
    CM.Node2d(2, "Node_3", L, L),
    CM.Node2d(3, "Node_4", 2*L,2*L),
    CM.Node2d(4, "Node_5", 2*L, L),
    CM.Node2d(5, "Node_6", 3*L, 2*L),
    CM.Node2d(6, "Node_7", 3*L, L),
    CM.Node2d(7, "Node_8", 3*L, 0),
    CM.Node2d(8, "Node_9", 4*L, 2*L),
    CM.Node2d(9, "Node_10", 4*L, L),
    CM.Node2d(10, "Node_11", 4*L, 0),
    CM.Node2d(11, "Node_12", 5*L, 2*L),
    CM.Node2d(12, "Node_13", 5*L, L),
    CM.Node2d(13, "Node_14", 6*L, 2*L),
    CM.Node2d(14, "Node_15", 6*L, L),
    CM.Node2d(15, "Node_16", 7*L, L),
]

beamconnections = [[1,2,S],[1,3,S],[2,3,S],[2,4,S],[3,4,S],[3,5,S],[4,5,S],
 [4,6,S],[5,6,S],[5,7,S],[6,7,S],[6,9,S],[6,10,S],[7,8,S],[7,9,S],[7,10,S],
 [8,10,S],[9,10,S],[9,12,S],[9,13,S],[6,10,S],[7,8,S],[7,9,S],[7,10,S],[7,11,S],
 [9,10,S],[9,12,S],[9,13,S],[10,11,S],[10,13,S],[12,13,S],[12,14,S],[12,15,S],
 [13,15,S],[14,15,S],[14,16,S],[15,16,S]
 ]

BeamList = []
i=0
for connection in beamconnections:
    BeamList.append(CM.Edge(
        i, 
        "Beam_{0}^{1}".format(
            connection[0],
            connection[1]), 
        Nodes[connection[0]-1], 
        Nodes[connection[1]-1], 
        connection[2]))
    i+=1


Set up which forces and displacements you know


In [4]:
knownDisplacements = [CM.NodeValue(Nodes[0], CM.Axis.X, 0),
                      CM.NodeValue(Nodes[0], CM.Axis.Y, 0),
                      CM.NodeValue(Nodes[7], CM.Axis.X, 0),
                      CM.NodeValue(Nodes[7], CM.Axis.Y, 0),
                      CM.NodeValue(Nodes[10], CM.Axis.X, 0),
                      CM.NodeValue(Nodes[10], CM.Axis.Y, 0),
                      CM.NodeValue(Nodes[15], CM.Axis.X, 0),
                      CM.NodeValue(Nodes[15], CM.Axis.Y, 0),]

F1 = sp.Symbol('F_1')
F2 = sp.Symbol('F_2')
F3 = sp.Symbol('F_3')
alpha = sp.Symbol("\\alpha")

knownForces = [CM.NodeValue(Nodes[1], CM.Axis.X, 0),
               CM.NodeValue(Nodes[1], CM.Axis.Y, 0),
               CM.NodeValue(Nodes[2], CM.Axis.X, 0),
               CM.NodeValue(Nodes[2], CM.Axis.Y, -F1),
               CM.NodeValue(Nodes[3], CM.Axis.X, 0),
               CM.NodeValue(Nodes[3], CM.Axis.Y, 0),
               CM.NodeValue(Nodes[4], CM.Axis.X, 0),
               CM.NodeValue(Nodes[4], CM.Axis.Y, -F1),
               CM.NodeValue(Nodes[5], CM.Axis.X, sp.cos(alpha)*F3),
               CM.NodeValue(Nodes[5], CM.Axis.Y, -sp.sin(alpha)*F3),
               CM.NodeValue(Nodes[6], CM.Axis.X, 0),
               CM.NodeValue(Nodes[6], CM.Axis.Y, 0),
               CM.NodeValue(Nodes[8], CM.Axis.X, 0),
               CM.NodeValue(Nodes[8], CM.Axis.Y, 0),
               CM.NodeValue(Nodes[9], CM.Axis.X, 0),
               CM.NodeValue(Nodes[9], CM.Axis.Y, 0),
               CM.NodeValue(Nodes[11], CM.Axis.X, 0),
               CM.NodeValue(Nodes[11], CM.Axis.Y, 0),
               CM.NodeValue(Nodes[12], CM.Axis.X, 0),
               CM.NodeValue(Nodes[12], CM.Axis.Y, -F2),
               CM.NodeValue(Nodes[13], CM.Axis.X, 0),
               CM.NodeValue(Nodes[13], CM.Axis.Y, 0),
               CM.NodeValue(Nodes[14], CM.Axis.X, 0),
               CM.NodeValue(Nodes[14], CM.Axis.Y, -F2)
    ]


Set up the system


In [5]:
System = CM.DlesSolver2D(Nodes, BeamList)
sp.simplify(System.SystemMatrix*L/S)


Matrix([
[sqrt(2)/4 + 1,  sqrt(2)/4,    -sqrt(2)/4,    -sqrt(2)/4,            -1,             0,             0,             0,             0,             0,               0,               0,               0,               0,          0,             0,           0,           0,               0,               0,          0,             0,             0,             0,             0,             0,             0,             0,             0,             0,             0,          0],
[    sqrt(2)/4,  sqrt(2)/4,    -sqrt(2)/4,    -sqrt(2)/4,             0,             0,             0,             0,             0,             0,               0,               0,               0,               0,          0,             0,           0,           0,               0,               0,          0,             0,             0,             0,             0,             0,             0,             0,             0,             0,             0,          0],
[   -sqrt(2)/4, -sqrt(2)/4, sqrt(2)

Give the known displacements and forces to the split function. The system will subdivide main matrix into the matrixes Kcc, Koc, Kco and Koo for solving the Discrete Linear Elastic System.

If the matrix is large with symbols, it's advised to set the parameter "invert" to False as shown here and assign Kcc_inv yourself. Otherwise remove or set the optional parameter to True. Do note that inversion is often a expensive calculation using symbolic methodes. Hence why in the example below covert it to numeric before inversion.


In [6]:
System.SplitSystemMatrix(knownDisplacements, knownForces, False)
System.Kcc_inv = sp.N(System.Kcc*(L/S)).inv()*(L/S)
sp.N(System.Kcc_inv*(S/L),3)

Matrix([
[    1.81,   -1.14,    0.344,  -0.908,      1.05,  -0.945,  0.452,  -0.708,   0.524,  -0.111,   0.323, -0.00576,   0.366, -0.0801,   0.328,   -0.122,   0.305, -0.0212,   0.279, -0.0815,   0.245,  0.0742,    0.17,    0.0139],
[   -1.14,    3.08,   -0.402,    2.76,    -0.461,    1.93, -0.488,    1.61, -0.0922,   0.323,  -0.259,    0.087,  -0.119,  0.0322,  -0.235,   0.0677,  -0.108, -0.0429,  -0.168, -0.0319, -0.0973, -0.0662, -0.0894,   -0.0552],
[   0.344,  -0.402,    0.735,  -0.423,     0.324, -0.0697,   0.49, -0.0902,   0.283,  0.0586,   0.265,   0.0349,   0.225,  0.0249,   0.211, -0.00841,   0.194,  0.0472,   0.172,  0.0161,   0.163,  0.0744,   0.102,    0.0433],
[  -0.908,    2.76,   -0.423,    3.42,    -0.252,    2.28, -0.501,    1.93,  0.0605,   0.398,  -0.236,    0.116, -0.0322,  0.0153,  -0.203,   0.0487, -0.0386, -0.0656,  -0.129, -0.0721, -0.0451, -0.0634, -0.0611,   -0.0698],
[    1.05,  -0.461,    0.324,  -0.252,      1.26,  -0.597,  0.439,  -0.388,   0.677, -0.035

Supply the solver with a additional forces that are directly set on the outside of the system of the system.


In [7]:
knownDirectNodeForces = sp.zeros(len(knownDisplacements),1)
knownDirectNodeForces[0] = -F1
knownDirectNodeForces[7] = -F2
dc, f0, fr = System.Solve(knownDirectNodeForces)
sp.simplify(sp.N(dc*E*A/(L), 3))


Matrix([
[     1.62*F_1 + 0.0677*F_2 + 0.111*F_3*sin(\alpha) + 0.524*F_3*cos(\alpha)],
[   -4.38*F_1 + 0.0871*F_2 - 0.323*F_3*sin(\alpha) - 0.0922*F_3*cos(\alpha)],
[   0.513*F_1 - 0.0593*F_2 - 0.0586*F_3*sin(\alpha) + 0.283*F_3*cos(\alpha)],
[    -5.35*F_1 + 0.142*F_2 - 0.398*F_3*sin(\alpha) + 0.0605*F_3*cos(\alpha)],
[     0.64*F_1 + 0.122*F_2 + 0.0359*F_3*sin(\alpha) + 0.677*F_3*cos(\alpha)],
[    -5.41*F_1 + 0.115*F_2 - 0.705*F_3*sin(\alpha) + 0.0986*F_3*cos(\alpha)],
[      1.0*F_1 - 0.173*F_2 - 0.0422*F_3*sin(\alpha) + 0.413*F_3*cos(\alpha)],
[       -5.39*F_1 + 0.17*F_2 - 0.78*F_3*sin(\alpha) + 0.251*F_3*cos(\alpha)],
[    -0.312*F_1 + 0.232*F_2 - 0.114*F_3*sin(\alpha) + 0.982*F_3*cos(\alpha)],
[    -1.18*F_1 - 0.0808*F_2 - 0.921*F_3*sin(\alpha) + 0.114*F_3*cos(\alpha)],
[    0.466*F_1 - 0.342*F_2 + 0.0494*F_3*sin(\alpha) + 0.391*F_3*cos(\alpha)],
[  -0.353*F_1 - 0.0967*F_2 - 0.279*F_3*sin(\alpha) + 0.0804*F_3*cos(\alpha)],
[  -0.0413*F_1 + 0.467*F_2 - 0.0558*F_3*sin(\alpha) + 0

In [8]:
sp.N(sp.cancel(f0), 3)


Matrix([
[  0.463*F_1 + 0.00461*F_2 + 0.134*F_3*sin(\alpha) - 0.436*F_3*cos(\alpha)],
[  0.976*F_1 - 0.0547*F_2 + 0.0751*F_3*sin(\alpha) - 0.153*F_3*cos(\alpha)],
[-0.0964*F_1 + 0.364*F_2 + 0.0229*F_3*sin(\alpha) - 0.0874*F_3*cos(\alpha)],
[     0.61*F_1 + 0.558*F_2 + 0.582*F_3*sin(\alpha) - 0.248*F_3*cos(\alpha)],
[    -0.29*F_1 + 0.0868*F_2 - 0.116*F_3*sin(\alpha) - 0.11*F_3*cos(\alpha)],
[    0.447*F_1 + 0.535*F_2 + 0.351*F_3*sin(\alpha) + 0.288*F_3*cos(\alpha)],
[  -0.077*F_1 - 0.456*F_2 - 0.0404*F_3*sin(\alpha) - 0.367*F_3*cos(\alpha)],
[-0.0331*F_1 + 0.963*F_2 - 0.00819*F_3*sin(\alpha) + 0.113*F_3*cos(\alpha)]])

In [9]:
sp.N(sp.cancel(fr), 3)


Matrix([
[   1.46*F_1 + 0.00461*F_2 + 0.134*F_3*sin(\alpha) - 0.436*F_3*cos(\alpha)],
[  0.976*F_1 - 0.0547*F_2 + 0.0751*F_3*sin(\alpha) - 0.153*F_3*cos(\alpha)],
[-0.0964*F_1 + 0.364*F_2 + 0.0229*F_3*sin(\alpha) - 0.0874*F_3*cos(\alpha)],
[     0.61*F_1 + 0.558*F_2 + 0.582*F_3*sin(\alpha) - 0.248*F_3*cos(\alpha)],
[    -0.29*F_1 + 0.0868*F_2 - 0.116*F_3*sin(\alpha) - 0.11*F_3*cos(\alpha)],
[    0.447*F_1 + 0.535*F_2 + 0.351*F_3*sin(\alpha) + 0.288*F_3*cos(\alpha)],
[  -0.077*F_1 - 0.456*F_2 - 0.0404*F_3*sin(\alpha) - 0.367*F_3*cos(\alpha)],
[ -0.0331*F_1 + 1.96*F_2 - 0.00819*F_3*sin(\alpha) + 0.113*F_3*cos(\alpha)]])

The Solver supplies the beams their stretch and internal force so you can print the result using "LatexPrintResult"


In [10]:
def LatexString(beam: CM.Edge) -> str:
        return "${0}$ & Strech : ${1}$ & BeamForce : ${2}$".format(
            beam.Name, 
            sp.latex(sp.simplify(sp.N(beam.Stretch, 3))),
            sp.latex(sp.simplify(sp.N(beam.Stress, 3)))
            )

printstring = ""
for item in BeamList:
    printstring += LatexString(item) +'\n'
print(printstring)


$Beam_1^2$ & Strech : $\frac{L \left(- 1.95 F_{1} + 0.109 F_{2} - 0.15 F_{3} \sin{\left(\alpha \right)} + 0.305 F_{3} \cos{\left(\alpha \right)}\right)}{A E}$ & BeamForce : $- 1.38 F_{1} + 0.0774 F_{2} - 0.106 F_{3} \sin{\left(\alpha \right)} + 0.216 F_{3} \cos{\left(\alpha \right)}$
$Beam_1^3$ & Strech : $\frac{L \left(0.513 F_{1} - 0.0593 F_{2} - 0.0586 F_{3} \sin{\left(\alpha \right)} + 0.283 F_{3} \cos{\left(\alpha \right)}\right)}{A E}$ & BeamForce : $0.513 F_{1} - 0.0593 F_{2} - 0.0586 F_{3} \sin{\left(\alpha \right)} + 0.283 F_{3} \cos{\left(\alpha \right)}$
$Beam_2^3$ & Strech : $\frac{L \left(0.976 F_{1} - 0.0547 F_{2} + 0.0751 F_{3} \sin{\left(\alpha \right)} - 0.153 F_{3} \cos{\left(\alpha \right)}\right)}{A E}$ & BeamForce : $0.976 F_{1} - 0.0547 F_{2} + 0.0751 F_{3} \sin{\left(\alpha \right)} - 0.153 F_{3} \cos{\left(\alpha \right)}$
$Beam_2^4$ & Strech : $\frac{L \left(- 0.976 F_{1} + 0.0547 F_{2} - 0.0751 F_{3} \sin{\left(\alpha \right)} + 0.153 F_{3} \cos{\left(\alpha \